In [ ]:
from google import genai
import base64
import os
from google.genai import types
import mimetypes

In [ ]:
PROJECT_ID = "image-analysis-460808"  
REGION = "us-central1" 
generate_content_config = types.GenerateContentConfig(
    response_mime_type="application/json",
    temperature=0.0,
    max_output_tokens=8100,
    top_p=0.95,
    candidate_count=1,
)

In [ ]:
client = genai.Client(vertexai=True, project=PROJECT_ID, location=REGION)

In [ ]:
response_format = """ 
my response json format is: 
{
    "verdict": true or false,
    "confidence": 0.0 to 1.0,
    "reasoning": "Concise explanation of your decision",
    "is_employee": true or false or "uncertain",
    "is_employee_reasoning": "Brief explanation for employee status"
}
"""

In [ ]:
root_folder = "./false/false"

# Traverse all subfolders
for subdir, _, files in os.walk(root_folder):
    # Filter image files
    image_files = [os.path.join(subdir, f) for f in files if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

    # Ensure there are at least two images
    if len(image_files) >= 2:
        try:
            # Encode first two images
            encoded_image1 = base64.b64encode(open(image_files[0], "rb").read()).decode('utf-8')
            encoded_image2 = base64.b64encode(open(image_files[1], "rb").read()).decode('utf-8')
            mime_type1, _ = mimetypes.guess_type(image_files[0])
            mime_type2, _ = mimetypes.guess_type(image_files[1])

            # Create parts
            image_part1 = types.Part.from_bytes(
                mime_type=mime_type1,
                data=base64.b64decode(encoded_image1)
            )
            image_part2 = types.Part.from_bytes(
                mime_type=mime_type2,
                data=base64.b64decode(encoded_image2)
            )

            # Create content
            contents = [
                types.Content(
                    role="user",
                    parts=[
                        image_part1,
                        image_part2,
                        types.Part.from_text(text=response_format)
                    ],
                ),
            ]

            # Call the model
            response = client.models.generate_content(
                model="projects/237800639406/locations/us-central1/endpoints/9065478868571258880",
                contents=contents,
                config=generate_content_config,
            )

            # Print result
            print(f"\nResponse for folder: {subdir}")
            print(response.text)
            # Write to result.txt
            with open("./false/result.txt", "a", encoding="utf-8") as f:
                f.write(f"\n=== Response for folder: {subdir} ===\n")
                f.write(response.text + "\n")

        except Exception as e:
            print(f"Error processing {subdir}: {e}")

In [ ]:
folder_path = "./true/true/customer1"

# Get all file paths in the folder
image_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

# Ensure you have at least two images
if len(image_files) >= 2:
    encoded_image1 = base64.b64encode(open(image_files[0], "rb").read()).decode('utf-8')
    encoded_image2 = base64.b64encode(open(image_files[1], "rb").read()).decode('utf-8')


mime_type1, _ = mimetypes.guess_type(image_files[0])
print(mime_type1)
# Create parts for the images
image_part1 = types.Part.from_bytes(
    mime_type="image/png",
    data=base64.b64decode(encoded_image1)
)

image_part2 = types.Part.from_bytes(
    mime_type="image/png", 
    data=base64.b64decode(encoded_image2)
)
contents = [
    types.Content(
        role="user",
        parts=[
            image_part1,
            image_part2,
            types.Part.from_text(text=response_format)
        ],
    ),
]
response = client.models.generate_content(
    model="projects/237800639406/locations/us-central1/endpoints/9065478868571258880",
    contents=contents,
    config=generate_content_config,
)
# Print the model's response
print(response.text)
